# Estandarizacion de datos de los Anuarios Geoestadísticos de INEGI 2017

## 1. Introduccion
Parámetros que se obtienen de esta fuente:

ID |Descripción
---|:----------
P0610|Ventas de electricidad
P0701|Longitud total de la red de carreteras del municipio (excluyendo las autopistas)

In [1]:
descripciones = {
    'P0610': 'Ventas de electricidad',
    'P0701': 'Longitud total de la red de carreteras del municipio (excluyendo las autopistas)'
}

In [2]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import csv
import zipfile

In [3]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


## 2. Descarga de datos
Cada entidad cuenta con una página que presenta sus respectivos anuarios geoestadísticos. La manera más rápida de obtener las ligas de los anuarios es entrar a la biblioteca de INEGI (http://www.beta.inegi.org.mx/app/publicaciones/) y buscar la palabra "Anuario" en el campo de búsqueda.

CVE_EDO |Nombre|URL
---|:---|:----------
01|Aguascalientes|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092078
02|Baja California|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094874
03|Baja California Sur|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094881
04|Campeche|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095109
05|Coahuila de Zaragoza|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095406
06|Colima|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092061
07|Chiapas|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094836
08|Chihuahua|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092139
09|Ciudad de México|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094683
10|Durango|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092115
11|Guanajuato|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092146
12|Guerrero|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094690
13|Hidalgo|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095093
14|Jalisco|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092085
15|México|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094706
16|Michoacán de Ocampo|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092092
17|Morelos|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094713
18|Nayarit|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092054
19|Nuevo León|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094911
20|Oaxaca|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094843
21|Puebla|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094973
22|Querétaro|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092108
23|Quintana Roo|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095130
24|San Luis Potosí|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092122
25|Sinaloa|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094898
26|Sonora|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094904
27|Tabasco|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095123
28|Tamaulipas|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094928
29|Tlaxcala|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825096212
30|Veracruz de Ignacio de la Llave|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094980
31|Yucatán|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095116
32|Zacatecas|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092047

Dentro de cada página, se incluye una liga directa para descargar un archivo comprimido con las tablas de datos de cada anuario geoestadítico. La lista links contiene estas URL y se utilizará para sistematizar la descarga de datos.

In [4]:
raiz = 'http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/anuarios_2017/'
# El diccionario tiene como llave la CVE_EDO y dirige hacia la liga de descarga del archivo zip con las tablas del 
# Anuario Geoestadístico de cada estado
links = {
    '01': raiz + '702825092078.zip',
    '02': raiz + '702825094874.zip',
    '03': raiz + '702825094881.zip',
    '04': raiz + '702825095109.zip',
    '05': raiz + '702825095406.zip',
    '06': raiz + '702825092061.zip',
    '07': raiz + '702825094836.zip',
    '08': raiz + '702825092139.zip',
    '09': raiz + '702825094683.zip',
    '10': raiz + '702825092115.zip',
    '11': raiz + '702825092146.zip',
    '12': raiz + '702825094690.zip',
    '13': raiz + '702825095093.zip',
    '14': raiz + '702825092085.zip',
    '15': raiz + '702825094706.zip',
    '16': raiz + '702825092092.zip',
    '17': raiz + '702825094713.zip',
    '18': raiz + '702825092054.zip',
    '19': raiz + '702825094911.zip',
    '20': raiz + '702825094843.zip',
    '21': raiz + '702825094973.zip',
    '22': raiz + '702825092108.zip',
    '23': raiz + '702825095130.zip',
    '24': raiz + '702825092122.zip',
    '25': raiz + '702825094898.zip',
    '26': raiz + '702825094904.zip',
    '27': raiz + '702825095123.zip',
    '28': raiz + '702825094928.zip',
    '29': raiz + '702825096212.zip',
    '30': raiz + '702825094980.zip',
    '31': raiz + '702825095116.zip',
    '32': raiz + '702825092047.zip'
}

In [5]:
# Descarga de archivos a carpeta local
destino = r'D:\PCCS\00_RawData\01_CSV\AGEO\2017'
archivos = {}   # Diccionario para guardar memoria de descarga
for k,v in links.items():
    archivo_local = destino + r'\{}.zip'.format(k)
    if os.path.isfile(archivo_local):
        print('Ya existe el archivo: {}'.format(archivo_local))
        archivos[k] = archivo_local
    else:
        print('Descargando {} ... ... ... ... ... '.format(archivo_local))
        urllib.request.urlretrieve(v, archivo_local) #
        archivos[k] = archivo_local
        print('se descargó {}'.format(archivo_local))

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\01.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\02.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\03.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\04.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\05.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\06.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\07.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\08.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\09.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\10.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\11.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\12.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\13.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\14.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\AGEO\2017\15.zip
Ya existe el archivo: D:\

In [6]:
# Descompresión de archivos de estado
unzipped = {}
for estado, comprimido in archivos.items():
    target = destino + '\\' + estado
    if os.path.isdir(target):
        print('Ya existe el directorio: {}'.format(target))
        unzipped[estado] = target
    else:
        print('Descomprimiendo {} ... ... ... ... ... '.format(target))
        descomprimir = zipfile.ZipFile(comprimido, 'r')
        descomprimir.extractall(target)
        descomprimir.close
        unzipped[estado] = target
        

Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\01
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\02
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\03
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\04
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\05
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\06
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\07
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\08
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\09
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\10
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\11
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\12
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\13
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\14
Ya existe el directorio: D:\PCCS\00_RawData\01_CSV\AGEO\2017\15
Ya existe el directorio: D:\PCCS\00_RawD

### Extraccion de indices
Conocer la información que contiene cada hoja del índice geoestadístico puede ser muy valioso y es necesario para hacer una función que itere adecuadamente por los archivos de todos los estados, debido a que los anuarios geoestadísticos de cada estado tienen ligeras variaciones que impiden una iteración directa.

In [7]:
unzipped

{'01': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\01',
 '02': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\02',
 '03': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\03',
 '04': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\04',
 '05': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\05',
 '06': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\06',
 '07': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\07',
 '08': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\08',
 '09': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\09',
 '10': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\10',
 '11': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\11',
 '12': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\12',
 '13': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\13',
 '14': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\14',
 '15': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\15',
 '16': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\16',
 '17': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\17',
 '18': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\18',
 '19': 'D:

In [8]:
# Extraer indices
indices = {}
for estado, ruta in unzipped.items():
    for file in os.listdir(ruta):
        if file.endswith('.xls'):
            path = ruta + '\\' + file
            indice = pd.read_excel(path, sheetname='Índice', skiprows = 1)   # Primera lectura al indice para sacar columnas
            dtypes = list(indice)
            tempdic = {}
            for i in dtypes:
                tempdic[i] = 'str'
            indice = pd.read_excel(path, 
                                   sheetname='Índice', 
                                   skiprows = 1,
                                   dtype = tempdic).dropna(how = 'all')   # Segunda lectura al indice ya con dtypes
            name = list(indice)[0]           # Guarda el nombre del indice
            cols = []
            for i in range(len(list(indice))):
                cols.append('col{}'.format(i))    # Crea nombres estandar de columna
            indice.columns = cols                 # Asigna nombres de columna
            indice['indice'] = name
            indice['file'] = file
            if estado not in indices.keys():      # Crea un diccionario para cada estado, si no existe
                indices[estado] = {}
            indices[estado][name] = indice
            print('Procesado {} |||NOMBRE:||| {}; [{}]'.format(file, name, len(cols)))     # Imprime los resultados del proceso

Procesado Ags. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Ags. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Ags. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Ags. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Ags. 13 Ganadería.xls |||NOMBRE:||| 13. Ganadería; [3]
Procesado Ags. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Ags. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Ags. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Ags. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Ags. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Ags. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Ags. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Ags. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Ags. 21 Turismo.xls

Procesado Coah. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Coah. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Coah. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Coah. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Coah. 21 Turismo.xls |||NOMBRE:||| 21. Turismo; [3]
Procesado Coah. 22 Transportes y comunicaciones.xls |||NOMBRE:||| 22. Transportes y comunicaciones; [3]
Procesado Coah. 23 Indicadores financieros.xls |||NOMBRE:||| 23. Indicadores financieros; [3]
Procesado Coah. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Coah. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Procesado Coah. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado Coah. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
Procesado Coah. 5 Salud.xls |||NOMBRE:||| 5. Salud; [3]
Procesado Coah. 6 Educación, ciencia y tecnología.xls |||NOMBRE:||| 6. Educación, cienci

Procesado CDMX. 23 Sector externo.xls |||NOMBRE:||| 23. Sector externo; [3]
Procesado CDMX. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado CDMX. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
Procesado CDMX. 5 Salud.xls |||NOMBRE:||| 5. Salud; [3]
Procesado CDMX. 6 Educación, ciencia y tecnología.xls |||NOMBRE:||| 6. Educación, ciencia y tecnología; [3]
Procesado CDMX. 7 Cultura y deporte.xls |||NOMBRE:||| 7. Cultura y deporte; [3]
Procesado CDMX. 8 Gobierno.xls |||NOMBRE:||| 8. Gobierno; [3]
Procesado CDMX. 9 Seguridad y justicia.xls |||NOMBRE:||| 9. Seguridad y justicia; [3]
Procesado Dgo. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Dgo. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Dgo. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Dgo. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Dgo. 13 Ganadería.xls |||NOMBRE:||| 13. Gana

Procesado Jal. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Jal. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Jal. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Jal. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Jal. 13 Ganadería.xls |||NOMBRE:||| 13. Ganadería; [3]
Procesado Jal. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Jal. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Jal. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Jal. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Jal. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Jal. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Jal. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Jal. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Jal. 21 Turismo.xls

Procesado Nay. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Nay. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Nay. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Nay. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Nay. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Nay. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Nay. 21 Turismo.xls |||NOMBRE:||| 21. Turismo; [3]
Procesado Nay. 22 Transportes y comunicaciones.xls |||NOMBRE:||| 22. Transportes y comunicaciones; [3]
Procesado Nay. 23 Indicadores financieros.xls |||NOMBRE:||| 23. Indicadores financieros; [3]
Procesado Nay. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Nay. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Procesado Nay. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado Nay. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
P

Procesado Qro. 23 Indicadores financieros.xls |||NOMBRE:||| 23. Indicadores financieros; [3]
Procesado Qro. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Qro. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Procesado Qro. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado Qro. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
Procesado Qro. 5 Salud.xls |||NOMBRE:||| 5. Salud; [3]
Procesado Qro. 6 Educación, ciencia y tecnología.xls |||NOMBRE:||| 6. Educación, ciencia y tecnología; [3]
Procesado Qro. 7 Cultura y deporte.xls |||NOMBRE:||| 7. Cultura y deporte; [3]
Procesado Qro. 8 Gobierno.xls |||NOMBRE:||| 8. Gobierno; [3]
Procesado Qro. 9 Seguridad y justicia.xls |||NOMBRE:||| 9. Seguridad y justicia; [3]
Procesado Qroo. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Qroo. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Qroo. 11 Información económica agregada.xls |||NOMBR

Procesado Tab. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Tab. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Tab. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Tab. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Tab. 13 Ganadería.xls |||NOMBRE:||| 13. Ganadería; [3]
Procesado Tab. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Tab. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Tab. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Tab. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Tab. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Tab. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Tab. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Tab. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Tab. 21 Turismo.xls

Procesado Yuc. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Yuc. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Yuc. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Yuc. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Yuc. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Yuc. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Yuc. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Yuc. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Yuc. 21 Turismo.xls |||NOMBRE:||| 21. Turismo; [3]
Procesado Yuc. 22 Transportes y comunicaciones.xls |||NOMBRE:||| 22. Transportes y comunicaciones; [3]
Procesado Yuc. 23 Indicadores financieros.xls |||NOMBRE:||| 23. Indicadores financieros; [3]
Procesado Yuc. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Yuc. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Pro

In [9]:
# Reordenar los dataframes por tipo
indices_2 = {}
for estado in indices.keys():
    for indice in indices[estado].keys():
        if indice not in indices_2.keys():
            indices_2[indice] = {}
        indices_2[indice][estado] = indices[estado][indice]

In [11]:
# Convertir indices en archivos unicos.
finalindexes = {}
for i in indices_2.keys():
    print(i)
    frameslist = []
    for estado in indices_2[i].keys():
        frame = indices_2[i][estado]
        frame['estado'] = estado
        frameslist.append(frame)
    fullindex = pd.concat(frameslist)
    finalindexes[i] = fullindex
    print('Hecho: {}\n'.format(i))

1. Aspectos geográficos
Hecho: 1. Aspectos geográficos

10. Trabajo
Hecho: 10. Trabajo

11. Información económica agregada
Hecho: 11. Información económica agregada

12. Agricultura
Hecho: 12. Agricultura

13. Ganadería
Hecho: 13. Ganadería

14. Aprovechamiento forestal
Hecho: 14. Aprovechamiento forestal

15. Pesca
Hecho: 15. Pesca

16. Minería
Hecho: 16. Minería

17. Industria manufacturera
Hecho: 17. Industria manufacturera

18. Construcción
Hecho: 18. Construcción

19. Electricidad
Hecho: 19. Electricidad

2. Medio ambiente
Hecho: 2. Medio ambiente

20. Comercio
Hecho: 20. Comercio

21. Turismo
Hecho: 21. Turismo

22. Transportes y comunicaciones
Hecho: 22. Transportes y comunicaciones

23. Indicadores financieros
Hecho: 23. Indicadores financieros

24. Finanzas públicas
Hecho: 24. Finanzas públicas

25. Sector externo
Hecho: 25. Sector externo

3. Población
Hecho: 3. Población

4. Vivienda y urbanización
Hecho: 4. Vivienda y urbanización

5. Salud
Hecho: 5. Salud

6. Educación, ci

In [12]:
# Escribir archivos xlsx
path = r'D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices'
for indice in finalindexes.keys():
    file = path+'\\'+indice+'.xlsx'
    writer = pd.ExcelWriter(file)
    finalindexes[indice].to_excel(writer, sheet_name = 'Indice')
    writer.save()
    print('[{}] lineas - archivo {}'.format(len(finalindexes[indice]), file))

[1852] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\1. Aspectos geográficos.xlsx
[3343] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\10. Trabajo.xlsx
[896] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\11. Información económica agregada.xlsx
[540] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\12. Agricultura.xlsx
[1164] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\13. Ganadería.xlsx
[966] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\14. Aprovechamiento forestal.xlsx
[269] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\15. Pesca.xlsx
[275] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\16. Minería.xlsx
[169] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\17. Industria manufacturera.xlsx
[593] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\18. Construcción.xlsx
[822] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\19. Electrici

Los índices obtenidos de esta manera recibirán una limpieza manual desde Excel.

### Estandarizacion de datos para Parámetros.

**P0610 Ventas de electricidad**

Debido a la falta de estructura de los índices de parámetros de electricidad, tuvieron que ser estandarizados manualmente en excel. Con los índices estandarizados ya es posible generar un iterador 

In [13]:
# Importar dataset de índices
f_indice = r'D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\Limpios\Electricidad.xlsx'
ds_indices = pd.read_excel(f_indice, dtype={'Numeral':'str', 'estado':'str'}).set_index('estado')
ds_indices.head()

,Numeral,Titulo,indice,file,Date,Units
estado,,,,,,
09,17.1,"Centrales generadoras, unidades de generación,...",17. Electricidad,CDMX. 17 Electricidad.xls,2016,NaN
09,17.2,"Usuarios, volumen y valor de las ventas de ene...",17. Electricidad,CDMX. 17 Electricidad.xls,2016,NaN
09,17.3,Usuarios de energía eléctrica por delegación s...,17. Electricidad,CDMX. 17 Electricidad.xls,Al 31 de diciembre de 2016,NaN
09,17.4,Volumen de las ventas de energía eléctrica por...,17. Electricidad,CDMX. 17 Electricidad.xls,2016,(Megawatts-hora)
09,17.5,Valor de las ventas de energía eléctrica por d...,17. Electricidad,CDMX. 17 Electricidad.xls,Al 31 de diciembre de 2016,(Miles de pesos)


Seleccionar renglones correspondientes a volumen de ventas de energía en MW/h

In [14]:
# Dataframe con índice de hojas sobre el tema "Ventas de electricidad"
ventaselec = ds_indices[ds_indices['Units'] == '(Megawatts-hora)']
ventaselec.head()

,Numeral,Titulo,indice,file,Date,Units
estado,,,,,,
09,17.4,Volumen de las ventas de energía eléctrica por...,17. Electricidad,CDMX. 17 Electricidad.xls,2016,(Megawatts-hora)
01,19.3,Volumen de las ventas de energía eléctrica por...,19. Electricidad,Ags. 19 Electricidad.xls,2016,(Megawatts-hora)
02,19.4,Volumen de las ventas de energía eléctrica por...,19. Electricidad,Bc. 19 Electricidad.xls,2016,(Megawatts-hora)
03,19.4,Volumen de las ventas de energía eléctrica por...,19. Electricidad,Bcs. 19 Electricidad.xls,2016,(Megawatts-hora)
04,19.4,Volumen de las ventas de energía eléctrica por...,19. Electricidad,Cam. 19 Electricidad.xls,2016,(Megawatts-hora)


In [15]:
len(ventaselec)

32

In [16]:
# Crear columna con rutas
path = r'D:\PCCS\00_RawData\01_CSV\AGEO\2017'
ventaselec['path'] = path+'\\'+ventaselec.index+'\\'+ventaselec['file']

C:\Users\carlos.arana\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# Definir función para traer datos a python
unnameds = set(['Unnamed: '+str(i) for i in range(0, 50)])   # Lista 'Unnamed: x' de 0 a 50

def get_ventas(path, sheet, estado):
    temp = pd.ExcelFile(path)
    temp = temp.parse(sheet, header = 6).dropna(axis = 0, how='all').dropna(axis = 1, how='all')
    
    # Elimina las columnas unnamed
    dropplets = set(temp.columns).intersection(unnameds)
    temp = temp.drop(dropplets, axis = 1)
    temp = temp.dropna(axis = 0, how='all')
    temp = temp.reset_index().drop('index', axis = 1)
    
    # Identifica los últimos renglones, que no contienen datos
    col0 = temp.columns[0]        # Nombre de la columna 0, para usarlo en un chingo de lugares. Bueno 3
    try: tempnotas = temp[col0][temp[col0] == 'Nota:'].index[0]  # Para las hojas que terminan en 'Notas'
    except: tempnotas = temp[col0][temp[col0] == 'a/'].index[0]   # Para las hojas que terminan en 'a/'
    print(tempnotas)
    
    # Aparta los renglones después de "a/", para conocer la información que dejé fuera.
    trashes = temp.iloc[tempnotas:-1]
    
    # Elimina los renglones después de "a/"
    temp = temp.iloc[0:tempnotas]
    
    # Crear columna de estado y renombrar la primera columna para poder concatenar datframes más tarde.
    temp['CVE_EDO'] = estado
    temp = temp.rename(columns={col0:'NOM_MUN'})
    print(type(temp))
    
    return temp, trashes

**Extraer datos de todas las ciudades**

Por medio de una función que se aplica al dataframe ventaselec, obtenemos los datos de ventas de energía eléctrica para todos los estados. Esta función incluye lineas para verificar los datasets extraidos.

In [19]:
# Funcion para extraer datos
def getdata(serie, estado):
    path = serie['path']
    sheet = serie['Numeral']
    print('{}\n{}'.format('-'*30, path))     # Imprime la ruta hacia el archivo
    print('Hoja: {}'.format(sheet))          # Imprime el nombre de la hoja que se va a extraer
    trashes, temp = get_ventas(path, sheet, estado)
    print(temp.iloc[[0, -1]][temp.columns[0]])
    print(list(temp))
    print(('len = {}'.format(len(temp))))
    return trashes, temp

In [21]:
ventasdic = {}
trashesdic = {}
for estado in ventaselec.index:
    ventasdic[estado], trashesdic[estado] = getdata(ventaselec.loc[estado], estado)

------------------------------
D:\PCCS\00_RawData\01_CSV\AGEO\2017\09\CDMX. 17 Electricidad.xls
Hoja: 17.4
17
<class 'pandas.core.frame.DataFrame'>
17      Nota:
23    Fuente:
Name: Delegación, dtype: object
['Delegación', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios']
len = 7
------------------------------
D:\PCCS\00_RawData\01_CSV\AGEO\2017\01\Ags. 19 Electricidad.xls
Hoja: 19.3
12
<class 'pandas.core.frame.DataFrame'>
12      Nota:
18    Fuente:
Name: Municipio, dtype: object
['Municipio', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios']
len = 7
------------------------------
D:\PCCS\00_RawData\01_CSV\AGEO\2017\02\Bc. 19 Electricidad.xls
Hoja: 19.4
6
<class 'pandas.core.frame.DataFrame'>
6       Nota:
12    Fuente:
Name: Municipio, dtype: object
['Municipio', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables

73
<class 'pandas.core.frame.DataFrame'>
73      Nota:
80    Fuente:
Name: Municipio, dtype: object
['Municipio', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'd/', 'Industrial y \nde servicios']
len = 8
------------------------------
D:\PCCS\00_RawData\01_CSV\AGEO\2017\27\Tab. 19 Electricidad.xls
Hoja: 19.3
18
<class 'pandas.core.frame.DataFrame'>
18      Nota:
24    Fuente:
Name: Municipio, dtype: object
['Municipio', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios']
len = 7
------------------------------
D:\PCCS\00_RawData\01_CSV\AGEO\2017\28\Tamps. 19 Electricidad.xls
Hoja: 19.4
45
<class 'pandas.core.frame.DataFrame'>
45      Nota:
51    Fuente:
Name: Municipio, dtype: object
['Municipio', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios']
len = 7
------------------------------
D:\PC

In [22]:
# Ejemplo de uno de los dataframes extraidos
ventasdic['09']

,NOM_MUN,Total,Doméstico,Alumbrado público,Bombeo de aguas potables y negras,Agrícola,Industrial y de servicios,CVE_EDO
0,Ciudad de México,14500021.99,3260893.80,647077.29,409572.90,737.0,10181741.0,09
1,Álvaro Obregón,1053345.40,277293.30,49974.58,10039.52,0.0,716038.0,09
2,Azcapotzalco,932103.88,169024.59,25533.89,11092.40,0.0,726453.0,09
3,Benito Juárez,1155190.91,261104.20,45049.06,18201.65,0.0,830836.0,09
4,Coyoacán,1049963.15,269165.37,56786.00,50700.78,0.0,673311.0,09
5,Cuajimalpa de Morelos,486602.73,109032.59,2994.41,20037.73,0.0,354538.0,09
6,Cuauhtémoc,1991862.94,234355.24,51756.96,3040.74,0.0,1702710.0,09
7,Gustavo A. Madero,1266887.71,393779.57,81768.85,7463.29,0.0,783876.0,09
8,Iztacalco,474297.60,144640.66,27044.60,4448.34,0.0,298164.0,09
9,Iztapalapa,1943814.92,494391.53,136657.64,81385.75,0.0,1231380.0,09


In [23]:
ventaselec['path']

estado
09    D:\PCCS\00_RawData\01_CSV\AGEO\2017\09\CDMX. 1...
01    D:\PCCS\00_RawData\01_CSV\AGEO\2017\01\Ags. 19...
02    D:\PCCS\00_RawData\01_CSV\AGEO\2017\02\Bc. 19 ...
03    D:\PCCS\00_RawData\01_CSV\AGEO\2017\03\Bcs. 19...
04    D:\PCCS\00_RawData\01_CSV\AGEO\2017\04\Cam. 19...
05    D:\PCCS\00_RawData\01_CSV\AGEO\2017\05\Coah. 1...
06    D:\PCCS\00_RawData\01_CSV\AGEO\2017\06\Col. 19...
07    D:\PCCS\00_RawData\01_CSV\AGEO\2017\07\Chis. 1...
08    D:\PCCS\00_RawData\01_CSV\AGEO\2017\08\Chih. 1...
10    D:\PCCS\00_RawData\01_CSV\AGEO\2017\10\Dgo. 19...
11    D:\PCCS\00_RawData\01_CSV\AGEO\2017\11\Gto. 19...
12    D:\PCCS\00_RawData\01_CSV\AGEO\2017\12\Gro. 19...
13    D:\PCCS\00_RawData\01_CSV\AGEO\2017\13\Hgo. 19...
14    D:\PCCS\00_RawData\01_CSV\AGEO\2017\14\Jal. 19...
15    D:\PCCS\00_RawData\01_CSV\AGEO\2017\15\Mex. 19...
16    D:\PCCS\00_RawData\01_CSV\AGEO\2017\16\Mich. 1...
17    D:\PCCS\00_RawData\01_CSV\AGEO\2017\17\Mor. 19...
18    D:\PCCS\00_RawData\01_CSV\AGEO\2017

**Revision de datos extraidos**

Revision de los datos que se dejaron fuera (en trashesdic), para asegurarme de que no dejé ningún municipio fuera.

In [24]:
a = '-'*30
for CVE_EDO in trashesdic.keys():
    print('{}\n{}\n{}'.format(a, trashesdic[CVE_EDO], a))

------------------------------
   Delegación  Total  Doméstico  Alumbrado\npúblico  \
17      Nota:    NaN        NaN                 NaN   
18         a/    NaN        NaN                 NaN   
19         b/    NaN        NaN                 NaN   
20         c/    NaN        NaN                 NaN   
21         d/    NaN        NaN                 NaN   
22         e/    NaN        NaN                 NaN   
23    Fuente:    NaN        NaN                 NaN   

    Bombeo de aguas \npotables y negras  Agrícola  Industrial y \nde servicios  
17                                  NaN       NaN                          NaN  
18                                  NaN       NaN                          NaN  
19                                  NaN       NaN                          NaN  
20                                  NaN       NaN                          NaN  
21                                  NaN       NaN                          NaN  
22                                  NaN   

Revision de columnas en todos los datasets

In [25]:
for CVE_EDO in ventasdic.keys():
    variables = list(ventasdic[CVE_EDO])
    longitud = len(variables)        # Cuantas variables existen?
    longset = len(set(variables))    # Cuantas variables son distintas?
    print('{}{} [{} - {}]\n{}\n{}'.format(a, CVE_EDO, longitud, longset, variables, a))

------------------------------09 [8 - 8]
['NOM_MUN', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios', 'CVE_EDO']
------------------------------
------------------------------01 [8 - 8]
['NOM_MUN', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios', 'CVE_EDO']
------------------------------
------------------------------02 [8 - 8]
['NOM_MUN', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios', 'CVE_EDO']
------------------------------
------------------------------03 [8 - 8]
['NOM_MUN', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \npotables y negras', 'Agrícola', 'Industrial y \nde servicios', 'CVE_EDO']
------------------------------
------------------------------04 [8 - 8]
['NOM_MUN', 'Total', 'Doméstico', 'Alumbrado\npúblico', 'Bombeo de aguas \n

Podemos ver que la extracción de columnas no fue uniforme. Tomando como base la Ciudad de México, se revisarán los casos particulares:

In [26]:
varCDMX = set(list(ventasdic['09']))
varCDMX

{'Agrícola',
 'Alumbrado\npúblico',
 'Bombeo de aguas \npotables y negras',
 'CVE_EDO',
 'Doméstico',
 'Industrial y \nde servicios',
 'NOM_MUN',
 'Total'}

Los siguientes estados contienen columnas que no son estándar:

In [27]:
nostandar = [] # Lista de estados cuyas columnas no son estandar
for CVE_EDO in ventasdic.keys():
    varsedo = set(list(ventasdic[CVE_EDO]))
    diffs = varCDMX.symmetric_difference(varsedo)
    if len(diffs) != 0:
        print('{}{}\n{}\n{}'.format(a, CVE_EDO, diffs, a))
        nostandar.append(CVE_EDO)

------------------------------05
{'d/', 'Industrial y \nde servicios', 'Industrial y\nde servicios', 'a/', '\nc/', '\nb/', '\ne/'}
------------------------------
------------------------------10
{'d/'}
------------------------------
------------------------------12
{'a/'}
------------------------------
------------------------------16
{'d/'}
------------------------------
------------------------------18
{'d/'}
------------------------------
------------------------------26
{'d/'}
------------------------------
------------------------------29
{'d/', 'Industrial y \nde servicios', 'Industrial y\nde servicios', 'Bombeo de aguas\npotables y negras', 'a/', '\nc/', 'Bombeo de aguas \npotables y negras', '\nb/', '\ne/'}
------------------------------


Se van a revisar caso por caso los siguientes estados.

In [28]:
nostandar

['05', '10', '12', '16', '18', '26', '29']

**CVE_EDO 05**

In [29]:
REV_EDO = '05'
ventasdic[REV_EDO].head(6)

,NOM_MUN,Total,Doméstico,a/,Alumbrado público,b/,Bombeo de aguas potables y negras,c/,Agrícola,d/,Industrial y de servicios,e/,CVE_EDO
0,,,,,,,,,,,,,05
1,Estado,1.0473e+07,1.78825e+06,NaN,179482,NaN,37537.6,NaN,777533,NaN,7.69018e+06,NaN,05
2,Acuña,450005,145186,NaN,7801.54,NaN,2267.88,NaN,643.985,NaN,294106,NaN,05
3,Allende,49678.8,26211.9,NaN,1198.21,NaN,484.663,NaN,5323.03,NaN,16461,NaN,05
4,Barroterán,20795.9,14887.9,NaN,1585.09,NaN,100.88,NaN,892.89,NaN,3329.2,NaN,05
5,Castaños,30804.1,19150.9,NaN,1621.16,NaN,765.013,NaN,1714.98,NaN,7552.05,NaN,05


In [30]:
list(ventasdic[REV_EDO])

['NOM_MUN',
 'Total',
 'Doméstico',
 'a/',
 'Alumbrado\npúblico',
 '\nb/',
 'Bombeo de aguas \npotables y negras',
 '\nc/',
 'Agrícola',
 'd/',
 'Industrial y\nde servicios',
 '\ne/',
 'CVE_EDO']

In [31]:
drops = ['a/', '\nb/', '\nc/', 'd/', '\ne/']     # Nombres de las columnas que se van a eliminar
ventasdic['05'] = ventasdic['05'].drop(drops, axis = 1)

In [32]:
old_colnames = list(ventasdic[REV_EDO])
new_colnames = list(ventasdic['09'])
colnames = {i:j for i,j in zip(old_colnames,new_colnames)}
ventasdic[REV_EDO] = ventasdic[REV_EDO].rename(columns = colnames)     # Normalizacion

In [33]:
ventasdic['05'].head()

,NOM_MUN,Total,Doméstico,Alumbrado público,Bombeo de aguas potables y negras,Agrícola,Industrial y de servicios,CVE_EDO
0,,,,,,,,05
1,Estado,1.0473e+07,1.78825e+06,179482,37537.6,777533,7.69018e+06,05
2,Acuña,450005,145186,7801.54,2267.88,643.985,294106,05
3,Allende,49678.8,26211.9,1198.21,484.663,5323.03,16461,05
4,Barroterán,20795.9,14887.9,1585.09,100.88,892.89,3329.2,05


**CVE_EDO 10**

In [34]:
REV_EDO = '10'
ventasdic[REV_EDO].head(6)

,NOM_MUN,Total,Doméstico,Alumbrado público,Bombeo de aguas potables y negras,Agrícola,d/,Industrial y de servicios,CVE_EDO
0,Estado,3296262.971,592933.995,89980.376,34728.9,587614,NaN,1991006.067,10
1,Canatlán,23834.768,8508.304,989.945,1282.24,7950.78,NaN,5103.500,10
2,Canelas,863.597,593.983,124.638,0.866,0.903,NaN,143.207,10
3,Coneto de Comonfort,1251.711,941.979,128.195,NS,58.17,NaN,123.261,10
4,Cuencamé f/,51749.898,11218.681,3738.850,1133.22,8013.25,NaN,27645.901,10
5,Durango,884281.089,219351.884,33761.225,8821.14,15818.6,NaN,606528.233,10


In [35]:
list(ventasdic[REV_EDO])

['NOM_MUN',
 'Total',
 'Doméstico',
 'Alumbrado\npúblico',
 'Bombeo de aguas \npotables y negras',
 'Agrícola',
 'd/',
 'Industrial y \nde servicios',
 'CVE_EDO']

In [36]:
drops = ['d/']     # Nombres de las columnas que se van a eliminar
ventasdic[REV_EDO] = ventasdic[REV_EDO].drop(drops, axis = 1)

**CVE_EDO 12**

In [37]:
REV_EDO = '12'
ventasdic[REV_EDO].head(6)

,NOM_MUN,Total,Doméstico,a/,Alumbrado público,Bombeo de aguas potables y negras,Agrícola,Industrial y de servicios,CVE_EDO
0,Estado,3.02624e+06,1.35635e+06,NaN,123199,40893.9,6693.46,1.4991e+06,12
1,Acapulco de Juárez,1.25765e+06,431807,NaN,21569.2,13682.7,342.077,790247,12
2,Acatepec,6674.6,4926.2,NaN,1146.76,2.49,0,599.152,12
3,Ahuacuotzingo,5339.5,3785.57,NaN,976.745,3.312,0,573.866,12
4,Ajuchitlán del Progreso,17097.2,12799.8,NaN,1622.97,209.179,2.914,2462.34,12
5,Alcozauca de Guerrero,5256.97,4046.31,NaN,677.312,0,0,533.35,12


In [38]:
list(ventasdic[REV_EDO])

['NOM_MUN',
 'Total',
 'Doméstico',
 'a/',
 'Alumbrado\npúblico',
 'Bombeo de aguas \npotables y negras',
 'Agrícola',
 'Industrial y \nde servicios',
 'CVE_EDO']

In [39]:
drops = ['a/']     # Nombres de las columnas que se van a eliminar
ventasdic[REV_EDO] = ventasdic[REV_EDO].drop(drops, axis = 1)

**CVE_EDO 16**

In [41]:
REV_EDO = '16'
ventasdic[REV_EDO].head(6)

,NOM_MUN,Total,Doméstico,Alumbrado público,Bombeo de aguas potables y negras,Agrícola,d/,Industrial y de servicios,CVE_EDO
0,Estado,6810375.585,1662398.634,192716.730,105454.377,392576,NaN,4457230.124,16
1,Acuitzio,5412.759,2735.813,525.145,193.394,901.769,NaN,1056.638,16
2,Aguililla,19403.611,6161.783,770.111,199.477,9173.73,NaN,3098.513,16
3,Álvaro Obregón,14302.270,5814.693,632.292,624.154,2955.29,NaN,4275.837,16
4,Angamacutiro,10582.273,4315.811,881.121,473.761,3266.76,NaN,1644.823,16
5,Angangueo,3249.187,1837.643,366.370,21.725,0,NaN,1023.449,16


In [42]:
list(ventasdic[REV_EDO])

['NOM_MUN',
 'Total',
 'Doméstico',
 'Alumbrado\npúblico',
 'Bombeo de aguas \npotables y negras',
 'Agrícola',
 'd/',
 'Industrial y \nde servicios',
 'CVE_EDO']

In [43]:
drops = ['d/']     # Nombres de las columnas que se van a eliminar
ventasdic[REV_EDO] = ventasdic[REV_EDO].drop(drops, axis = 1)

**CVE_EDO 18**

In [44]:
REV_EDO = '18'
ventasdic[REV_EDO].head(6)

,NOM_MUN,Total,Doméstico,Alumbrado público,Bombeo de aguas potables y negras,Agrícola,d/,Industrial y de servicios,CVE_EDO
0,Estado,1582233.916,656520.028,38115.838,29723.977,13561.358,NaN,844312.715,18
1,Acaponeta,50970.420,22690.196,1316.462,476.596,543.925,NaN,25943.241,18
2,Ahuacatlán,10749.127,6273.794,391.386,280.442,653.962,NaN,3149.543,18
3,Amatlán de Cañas,10012.917,5295.400,431.327,615.458,1879.160,NaN,1791.572,18
4,Bahía de Banderas,519158.867,145011.987,9121.276,10696.325,2496.600,NaN,351832.679,18
5,Compostela,96804.858,46462.406,1889.043,4570.057,1006.421,NaN,42876.931,18


In [45]:
list(ventasdic[REV_EDO])

['NOM_MUN',
 'Total',
 'Doméstico',
 'Alumbrado\npúblico',
 'Bombeo de aguas \npotables y negras',
 'Agrícola',
 'd/',
 'Industrial y \nde servicios',
 'CVE_EDO']

In [46]:
drops = ['d/']     # Nombres de las columnas que se van a eliminar
ventasdic[REV_EDO] = ventasdic[REV_EDO].drop(drops, axis = 1)

**CVE_EDO 26**

In [47]:
REV_EDO = '26'
ventasdic[REV_EDO].head(6)

,NOM_MUN,Total,Doméstico,Alumbrado público,Bombeo de aguas potables y negras,Agrícola,d/,Industrial y de servicios,CVE_EDO
0,Estado,1.025362e+07,3904402.438,105177.104,153968.072,1186227.796,NaN,4903842.354,26
1,Aconchi,4.740577e+03,2729.406,135.093,396.553,361.925,NaN,1117.600,26
2,Agua Prieta,1.979518e+05,62721.872,1914.842,10821.555,1038.627,NaN,121454.918,26
3,Alamos,9.046326e+04,19835.829,834.140,776.354,41.839,NaN,68975.100,26
4,Altar,1.137915e+05,13442.925,504.275,2546.795,63375.163,NaN,33922.348,26
5,Arivechi,1.098762e+03,839.499,91.947,27.683,0.000,NaN,139.633,26


In [48]:
list(ventasdic[REV_EDO])

['NOM_MUN',
 'Total',
 'Doméstico',
 'Alumbrado\npúblico',
 'Bombeo de aguas \npotables y negras',
 'Agrícola',
 'd/',
 'Industrial y \nde servicios',
 'CVE_EDO']

In [49]:
drops = ['d/']     # Nombres de las columnas que se van a eliminar
ventasdic[REV_EDO] = ventasdic[REV_EDO].drop(drops, axis = 1)

**CVE_EDO 29**

In [50]:
REV_EDO = '29'
ventasdic[REV_EDO].head(6)

,NOM_MUN,Total,Doméstico,a/,Alumbrado público,b/,Bombeo de aguas potables y negras,c/,Agrícola,d/,Industrial y de servicios,e/,CVE_EDO
0,,,,,,,,,,,,,29
1,Estado,1.91102e+06,336643,NaN,62097.5,NaN,54982.6,NaN,18360.9,NaN,1.43893e+06,NaN,29
2,Acuamanala de Miguel Hidalgo,11794.7,1479.77,NaN,264.946,NaN,443.276,NaN,123.615,NaN,9483.08,NaN,29
3,Amaxac de Guerrero,5232.96,2972.74,NaN,306.999,NaN,539.767,NaN,93.063,NaN,1320.39,NaN,29
4,Apetatitlán de Antonio Carvajal,52555,4251.49,NaN,807.129,NaN,800.246,NaN,0,NaN,46696.2,NaN,29
5,Apizaco,314063,24338,NaN,2889.19,NaN,2508.17,NaN,73.532,NaN,284254,NaN,29


In [51]:
list(ventasdic[REV_EDO])

['NOM_MUN',
 'Total',
 'Doméstico',
 'a/',
 'Alumbrado\npúblico',
 '\nb/',
 'Bombeo de aguas\npotables y negras',
 '\nc/',
 'Agrícola',
 'd/',
 'Industrial y\nde servicios',
 '\ne/',
 'CVE_EDO']

In [52]:
drops = ['a/', '\nb/', '\nc/', 'd/', '\ne/']     # Nombres de las columnas que se van a eliminar
ventasdic[REV_EDO] = ventasdic[REV_EDO].drop(drops, axis = 1)

In [53]:
old_colnames = list(ventasdic[REV_EDO])
new_colnames = list(ventasdic['09'])
colnames = {i:j for i,j in zip(old_colnames,new_colnames)}
ventasdic[REV_EDO] = ventasdic[REV_EDO].rename(columns = colnames)     # Normalizacion

** Consolidacion de dataframe **

Todos los dataframes se guardarán como un solo archivo único y chingón.

In [54]:
# Unificacion de datos a un solo dataset
ventasDS = pd.DataFrame()
for CVE_EDO, dataframe in ventasdic.items():
    estado = ventasdic[CVE_EDO]['NOM_MUN'][0]
    print('Adjuntando {} - {} -----'.format(CVE_EDO, estado))
    ventasDS = ventasDS.append(dataframe)

Adjuntando 09 - Ciudad de México -----
Adjuntando 01 - Estado -----
Adjuntando 02 - Estado -----
Adjuntando 03 - Estado -----
Adjuntando 04 - Estado -----
Adjuntando 05 -   -----
Adjuntando 06 - 2015 R/ -----
Adjuntando 07 - Estado -----
Adjuntando 08 - Estado -----
Adjuntando 10 - Estado -----
Adjuntando 11 - Estado -----
Adjuntando 12 - Estado -----
Adjuntando 13 - 2015 R/ -----
Adjuntando 14 - Estado -----
Adjuntando 15 - Estado -----
Adjuntando 16 - Estado -----
Adjuntando 17 - Estado -----
Adjuntando 18 - Estado -----
Adjuntando 19 - Estado -----
Adjuntando 20 - Estado -----
Adjuntando 21 - Estado -----
Adjuntando 22 - Estado -----
Adjuntando 23 - Estado -----
Adjuntando 24 - Estado -----
Adjuntando 25 - Estado -----
Adjuntando 26 - Estado -----
Adjuntando 27 - Estado -----
Adjuntando 28 - Estado -----
Adjuntando 29 -   -----
Adjuntando 30 - Estado -----
Adjuntando 31 - Estado -----
Adjuntando 32 - Estado -----


In [55]:
# Nombre final para columnas, para evitar duplicidades con otros datasets 
# y para eliminar acentos que podrían causar problemas de encoding
colnames = {
'NOM_MUN':'NOM_MUN',
'Total':'Total ventas elec',
'Doméstico':'VE Domestico',
'Alumbrado\npúblico':'VE alumbrado publico',
'Bombeo de aguas \npotables y negras':'VE Bombeo agua potable y negra',
'Agrícola':'VE Agricola',
'Industrial y \nde servicios':'VE Industrial y servicios',
}

In [56]:
ventasDS = ventasDS.rename(columns=colnames)
ventasDS.head()

,NOM_MUN,Total ventas elec,VE Domestico,VE alumbrado publico,VE Bombeo agua potable y negra,VE Agricola,VE Industrial y servicios,CVE_EDO
0,Ciudad de México,1.45e+07,3.26089e+06,647077,409573,737,1.01817e+07,09
1,Álvaro Obregón,1.05335e+06,277293,49974.6,10039.5,0,716038,09
2,Azcapotzalco,932104,169025,25533.9,11092.4,0,726453,09
3,Benito Juárez,1.15519e+06,261104,45049.1,18201.7,0,830836,09
4,Coyoacán,1.04996e+06,269165,56786,50700.8,0,673311,09


In [57]:
# Metadatos
metadatos = {
    'Nombre del Dataset': 'Anuario Geoestadistico 2017 por estado, Datos de electricidad',
    'Descripcion del dataset': None,
    'Disponibilidad Temporal': '2016',
    'Periodo de actualizacion': 'Anual',
    'Nivel de Desagregacion': 'Municipal',
    'Notas': 's/n',
    'Fuente': 'INEGI - Anuarios Geoestadisticos',
    'URL_Fuente': 'http://www.beta.inegi.org.mx/proyectos/ccpv/2010/?#section',
    'Dataset base': None
}

metadatos = pd.DataFrame.from_dict(metadatos, orient='index', dtype='str')
metadatos.columns = ['Descripcion']
metadatos= metadatos.rename_axis('Metadato')
metadatos

,Descripcion
Metadato,
Nombre del Dataset,"Anuario Geoestadistico 2017 por estado, Datos ..."
Descripcion del dataset,None
Disponibilidad Temporal,2016
Periodo de actualizacion,Anual
Nivel de Desagregacion,Municipal
Notas,s/n
Fuente,INEGI - Anuarios Geoestadisticos
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/ccpv/20...
Dataset base,None


In [58]:
Variables = {
    'NOM_MUN':'Nombre del Municipio',
    'Total ventas elec':'Total de ventas de energía electrica',
    'VE Domestico':'Ventas de energia en el sector domestico (Megawatts-Hora)',
    'VE alumbrado publico':'Ventas de energia en alumbrado publico (Megawatts-Hora)',
    'VE Bombeo agua potable y negra':'Ventas de energia en bombeo de agua potable y negra (Megawatts-Hora)',
    'VE Agricola':'Ventas de energia en el sector Agricola (Megawatts-Hora)',
    'VE Industrial y servicios':'Ventas de energia en el sector industrial y de Servicios (Megawatts-Hora)',
    'CVE_EDO':'Clave Geoestadistica Estatal de 2 Digitos',
}

Variables = pd.DataFrame.from_dict(Variables, orient='index', dtype='str')
Variables.columns = ['Descripcion']
Variables= Variables.rename_axis('Mnemonico')
Variables

,Descripcion
Mnemonico,
NOM_MUN,Nombre del Municipio
Total ventas elec,Total de ventas de energía electrica
VE Domestico,Ventas de energia en el sector domestico (Mega...
VE alumbrado publico,Ventas de energia en alumbrado publico (Megawa...
VE Bombeo agua potable y negra,Ventas de energia en bombeo de agua potable y ...
VE Agricola,Ventas de energia en el sector Agricola (Megaw...
VE Industrial y servicios,Ventas de energia en el sector industrial y de...
CVE_EDO,Clave Geoestadistica Estatal de 2 Digitos


Como todo lo triste de esta vida, el dataset no tiene asignadas claves geoestadísticas municipales, por lo que será necesario etiquetarlo manualmente en Excel.

In [59]:
#Exportar dataset
file = r'D:\PCCS\01_Dmine\Datasets\AGEO\2017'+'\\'+'electricidad.xlsx'
writer = pd.ExcelWriter(file)
ventasDS.to_excel(writer, sheet_name = 'DATOS')
metadatos.to_excel(writer, sheet_name ='METADATOS')
Variables.to_excel(writer, sheet_name ='VARIABLES')
writer.save()

Las siguientes columnas de código muestran el dataset después de la limpieza realizada en Excel.

In [60]:
# Muestra del dataset con claves geoestadísticas asignadas
archivo = r'D:\PCCS\01_Dmine\Datasets\AGEO\2017\VentasElectricidad.xlsx'
elec = pd.read_excel(archivo, sheetname='DATOS', dtype={'CVE_MUN':'str'})
elec = elec.set_index('CVE_MUN')
elec.head()

,NOM_MUN,Total ventas elec,VE Domestico,VE alumbrado publico,VE Bombeo agua potable y negra,VE Agricola,VE Iustrial y servicios
CVE_MUN,,,,,,,
09010,Álvaro Obregón,1053345.40,277293.30,49974.58,10039.52,0.0,716038.0
09002,Azcapotzalco,932103.88,169024.59,25533.89,11092.40,0.0,726453.0
09014,Benito Juárez,1155190.91,261104.20,45049.06,18201.65,0.0,830836.0
09003,Coyoacán,1049963.15,269165.37,56786.00,50700.78,0.0,673311.0
09004,Cuajimalpa de Morelos,486602.73,109032.59,2994.41,20037.73,0.0,354538.0


El dataset limpio quedó guardado como '..\PCCS\01_Dmine\Datasets\AGEO\2017\VentasElectricidad.xlsx'

**P0701 Longitud total de la red de carreteras del municipio (excluyendo las autopistas)**

Debido a la falta de estructura de los índices de parámetros de Movilidad, tuvieron que ser estandarizados manualmente en excel. Con los índices estandarizados ya es posible generar un iterador 